# SI 370 - Class 10.2: Classification and Sentiment Analysis

## Objectives:
- Be able to perform classifications using the following methods and interpret their results
  - Decision Tree
  - Random Forest
- be aware of some text processing techniques and how to accomplish them using Python
- Be able to use MonkeyLearn to create, train, and test a sentiment analyser

We had a lot of difficulty using pydot-ng last class, so we're going to try an alternative.  First, we're going to install python-graphviz using something like:

```conda install python-graphviz
```

Note that you might need to specify the environment (conda install -n si370 python-graphviz)

Note that you might need to update scikit-learn if you didn't last class. Do either of the following:
  - `conda update scikit-learn`
  - `conda update -n si370 scikit-learn`
  
You might need to restart jupyter after you install and update those modules.

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import sklearn.ensemble as skens
import sklearn.metrics as skmetric
import sklearn.naive_bayes as sknb
import sklearn.tree as sktree
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='white', color_codes=True, font_scale=1.3)
import graphviz
import sklearn.externals.six as sksix
import IPython.display as ipd
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import os

# make the Pandas tables a little more readable
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))


In [2]:
# Filter all warnings.
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load the usual dataset, but in a different format
from sklearn.datasets import load_iris
from sklearn import tree
df_iris = load_iris()


Take a look at the data set that comes from scikit-learn: it's a bit different than the others we've seen.

In [4]:
print(df_iris)

{'target_names': array(['setosa', 'versicolor', 'virginica'],
      dtype='|S10'), 'data': array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5

# 1. Decision Trees

## 1.1 Train a decision tree classifier

We will train a decision tree classifier to classify iris species. Here are the specifications:
- __Objective__: predict which species an iris instance belongs to.
- __Possible classes__: "setosa", "versicolor", and "virginica"
- __Features__: all four features


In [5]:
## Build and train the classifier
dt_model = sktree.DecisionTreeClassifier(max_depth=2,
                                         criterion='entropy')
# given first 4 columns, learn the species
dt_model.fit(df_iris.data,df_iris.target)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

The next code block will create a file called iris1.pdf in the same directory as this notebook.  Use a viewer (like Preview) to examine the resuling file.

In [6]:
dot_data = sktree.export_graphviz(dt_model, out_file=None,
                                  feature_names=df_iris.feature_names,  
                                  class_names=df_iris.target_names,  
                                  filled=True, rounded=True,  
                                  special_characters=True)  
graph = graphviz.Source(dot_data) 
graph.render("iris1") 

'iris1.pdf'

![](iris1.png)

To interpret the above plot, please read through the following paragraphs quoted from http://www.thegrimmscientist.com/tutorial-decision-trees/, which has slightly different results than our data set because they used alightly different data.  Their results looks like:
![](http://www.thegrimmscientist.com/content/images/2017/06/TreeDepth1.png)
> This image needs a bit of explanation. The top node represents the whole dataset. The whole dataset has 150 samples, as seen in the third line of that first node. We'll explain entropy more later, but for now, think of it as a formalization of the amount of uncertainty in the data. The first line on the top node gives us the decision our tree magically found. When that decision "petal width is less than or equal to 0.80 cm" is true for a given sample, that sample is assigned to the bottom left node. When it is false, the sample is assigned to the bottom right node.

>On these two leaf nodes, the number of samples is shown on the second line. The number of samples broken down by class is on the third line. On the left leaf node, all 50 of the samples in class 1 and none from the other two classes are represented. Given the decision and the data at hand, we are certain to have a sample in class 1 (setosa). Since we are totally certain about that, uncertainty (entropy) is 0.

>When our petal width <= 0.80 decider is false, the given sample is assigned to the right node. It works out that all 50 samples from class 2 (versicolor) and all 50 samples from class 3 (virginica) are assigned to this right leaf. Thus, when we're at this leaf given the decision, we're evenly unsure whether we're in class 2 or class 3. The amount of uncertainty this works out to be is 1.0. Again, we'll explain the math and meaning behind that later.

Now use our classifier to predict the class of each of our data points

In [90]:
dt_model.predict(df_iris.data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [91]:
df_iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## 1.1 Training decision trees
1. Train a decision tree with `max_depth=2`. Plot it as a graph (same below).
2. Train a decision tree with `max_depth=3`.
3. Train a decision tree without `max_depth` limitation.


_NOTE_: see http://www.thegrimmscientist.com/tutorial-decision-trees/ for awesome visualizations and explanations to intuitively understand what decision tree algorithm is doing.

<font color="red">Modify the following cell to create a function that takes
a single parameter (depth) and uses that parameter to call the code above to 
set the max_depth attribute on the sktree.DecisionTreeClassifier.

In [92]:
### YOUR CODE HERE -- it might be easier to create a function that takes the desired max_depth as a 
### parameter

## 1.2 Visualizing a Decision Tree Classifier

We train a decision tree with `max_depth=2` to cluster the irises into three species.

In [93]:

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

# Parameters
n_classes = 3
plot_colors = "ryb"
plot_step = 0.02

# Load data
iris = load_iris()

for pairidx, pair in enumerate([[0, 1], [0, 2], [0, 3],
                                [1, 2], [1, 3], [2, 3]]):
    # We only take the two corresponding features
    X = iris.data[:, pair]
    y = iris.target

    # Train
    clf = DecisionTreeClassifier().fit(X, y)

    # Plot the decision boundary
    plt.subplot(2, 3, pairidx + 1)

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))
    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, cmap=plt.cm.RdYlBu)

    plt.xlabel(iris.feature_names[pair[0]])
    plt.ylabel(iris.feature_names[pair[1]])

    # Plot the training points
    for i, color in zip(range(n_classes), plot_colors):
        idx = np.where(y == i)
        plt.scatter(X[idx, 0], X[idx, 1], c=color, label=iris.target_names[i],
                    cmap=plt.cm.RdYlBu, edgecolor='black', s=15)

plt.suptitle("Decision surface of a decision tree using paired features")
plt.legend(loc='lower right', borderpad=0, handletextpad=0)
plt.axis("tight")
plt.show()

![](decisionsurface.png)

# 2. Random Forest

Let us build a random forest classifier to classify irises into three species.
- __Objective__: predict which species an iris instance belongs to.
- __Possible classes__: "setosa", "versicolor", and "virginica"
- __Features__: all four features

In [78]:
# build a random forest
rf_model = skens.RandomForestClassifier(n_estimators=10,oob_score=True)
rf_model.fit(df_iris.data,df_iris.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

Use the same approach that you did in predicting the Decision Tree Classifier.

In [81]:
# now do the predicting... use the same approach as you did in testing the Decision Tree Classifier above,
# but don't include the .sample() line
rf_model.predict(df_iris.data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [82]:
df_iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


# 3. Introduction to Text Processing

(some material from https://nlp.stanford.edu/IR-book)

# Text Processing
* an analytics pipline
* basic operations:
 * tokenization
 * stop words
 * normalization
 * stemming
 * lemmatization

# Tokenization

* splitting up text: documents -> paragraphs -> sentences -> words
* easy: string.split()?
![](frc.png)

![](arent.png)

![](oneill.png)

# Stopwords
* common words that don't add a lot of information to a document
![](stopwords.png)

# Stopwords: A cautionary note
* take a close look at the stopwords list that is commonly used in Python's NLTK package:

In [96]:
import nltk
print(sorted(nltk.corpus.stopwords.words('english')))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', 'd', 'did', 'didn', 'do', 'does', 'doesn', 'doing', 'don', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', 'has', 'hasn', 'have', 'haven', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', 'more', 'most', 'mustn', 'my', 'myself', 'needn', 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', 'she', 'should', 'shouldn', 'so', 'some', 'such', 't', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'v

* are all of these words really unimportant?

# Normalization
* token normalization is process of canonicalizing tokens
 * U.S.A. -> USA (-> usa ?)
 * anti-discriminatory -> antidiscriminatory
 * C.A.T -> ?

# Stemming
* stemming: chopping off the ends of words:
 * rule-based: 
   * SSES -> SS (caresses -> caress)
   * IES -> I (ponies -> poni)
   * SS -> SS (caress -> caress)
   * S -> (cats -> cat)
  * most common algorithm is _Porter stemmer_
 


# Lemmatization
* uses much more involved algorithms to convert each token to its lemma (canonical form of a word)
* e.g. ran, run, runs, running -> run
* e.g. is, are, am -> to be

# Stemming and Lemmatization: A cautionary note
* increases recall but lowers precision
* doesn't help a lot with most modern text analyses

# Part-of-speech tagging
* identification of different parts of speech (e.g. nouns, verbs, adjectives, etc.)
* based on corpus linguistics
* uses a corpus ("Brown corpus")

# Sentiment Analysis
* identify positive, negative (and sometimes neutral) sentiment in text
* relies on labelled data
* domain-specific (e.g. tweets are different from Yelp reviews)

# Text Processing in Python
* NLTK: Natural Language Toolkit
* Example: sentence tokenization

(some material from https://pythonspot.com/)

# Load the NLTK data (you only need to do this once)

In [109]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-dat

[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /Users/cteplovs/nltk_data...
[nltk_data]    |   Package shakespeare

True

# Sentence tokenization in Python


In [116]:
from nltk.tokenize import sent_tokenize, word_tokenize
 
data = "The School of Information delivers innovative, elegant and ethical solutions connecting people, information and technology. When there is a need for world-changing information discoveries, we will be there."
print(sent_tokenize(data))

['The School of Information delivers innovative, elegant and ethical solutions connecting people, information and technology.', 'When there is a need for world-changing information discoveries, we will be there.']


# Word tokenization and stopword removal in Python
* Example: tokenization and stopword removal:

In [117]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
 
data = "The School of Information delivers innovative, elegant and ethical solutions connecting people, information and technology. When there is a need for world-changing information discoveries, we will be there."
stopWords = set(stopwords.words('english'))
words = word_tokenize(data)
wordsFiltered = []
 
for w in words:
    if w not in stopWords:
        wordsFiltered.append(w)
 
print(wordsFiltered)

['The', 'School', 'Information', 'delivers', 'innovative', ',', 'elegant', 'ethical', 'solutions', 'connecting', 'people', ',', 'information', 'technology', '.', 'When', 'need', 'world-changing', 'information', 'discoveries', ',', '.']


# Stemming in Python

In [119]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
 
words = ["game","gaming","gamed","games"]
ps = PorterStemmer()
 
for word in words:
    print(word,ps.stem(word))

game game
gaming game
gamed game
games game


# Part-of-speech tagging in Python

In [113]:
import nltk
from nltk.tokenize import PunktSentenceTokenizer
 
document = 'Students in 370 are strongly advised to have taken at least one programming course and ideally a statistics course as well.'
sentences = nltk.sent_tokenize(document)   
for sent in sentences:
    print(nltk.pos_tag(nltk.word_tokenize(sent)))

[('Students', 'NNS'), ('in', 'IN'), ('370', 'CD'), ('are', 'VBP'), ('strongly', 'RB'), ('advised', 'VBN'), ('to', 'TO'), ('have', 'VB'), ('taken', 'VBN'), ('at', 'IN'), ('least', 'JJS'), ('one', 'CD'), ('programming', 'VBG'), ('course', 'NN'), ('and', 'CC'), ('ideally', 'RB'), ('a', 'DT'), ('statistics', 'NNS'), ('course', 'NN'), ('as', 'RB'), ('well', 'RB'), ('.', '.')]


![](nltk-speech-codes.png)

# Sentiment analysis in Python

In [114]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
positive_vocab = [ 'awesome', 'outstanding', 'fantastic', 'terrific', 'good', 'nice', 'great', ':)' ]
negative_vocab = [ 'bad', 'terrible','useless', 'hate', ':(' ]
neutral_vocab = [ 'movie','the','sound','was','is','actors','did','know','words','not' ]
 
positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]
neutral_features = [(word_feats(neu), 'neu') for neu in neutral_vocab]
 
train_set = negative_features + positive_features + neutral_features
 
classifier = NaiveBayesClassifier.train(train_set) 
 
# Predict
neg = 0
pos = 0
sentence = "Awesome movie, I liked it"
sentence = sentence.lower()
words = sentence.split(' ')
for word in words:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
    if classResult == 'pos':
        pos = pos + 1
 
print('Positive: ' + str(float(pos)/len(words)))
print('Negative: ' + str(float(neg)/len(words)))

Positive: 0.6
Negative: 0.2


# But that's getting a bit boring...

# So let's explore MonkeyLearn

## 3.1.  Obtain an MonkeyLearn API key

Go to the MonkeyLearn sign-up page (https://app.monkeylearn.com/accounts/register/) and register to get an API key.  To get your API token, click on your name after logging in, select "My Account", then "API keys" on the left side of the page.  Let's assign it to a variable so we don't have to keep pasting it into other cells:

In [115]:
MONKEYLEARN_API_KEY = '<YOUR TOKEN GOES HERE>'

## 3.2 Introduction to MonkeyLearn

MonkeyLearn is "a Machine Learning platform on the cloud that allows software companies and developers to easily extract actionable data from text."  It makes it easy to start investigating text classifiers.

Log into the system and take a look at the basic sentiment classifier at https://app.monkeylearn.com/main/classifiers/cl_Jx8qzYJh/

Paste some text into the box and click on "Classify".  Do the results seem reasonable?  Repeat it with some other text and see what you get.

You can also experiment with other classifiers that might be tuned to your problem space.  For example, the Tweet classifier might do a better job of classifying tweets.

## 3.3 Integrating with Python

You'll need to install the monkeylearn python module. The following code block should work:


In [ ]:
!pip install monkeylearn

Now try the following code, which is equivalent to trying out the generic Sentiment Analysis tool.
Don't forget to fill in your API key!

In [ ]:
from monkeylearn import MonkeyLearn

ml = MonkeyLearn(MONKEYLEARN_API_KEY)
text_list = ["This is the best sentiment analysis tool ever!!!"]
module_id = 'cl_Jx8qzYJh'
res = ml.classifiers.classify(module_id, text_list, sandbox=False)
print(res.result)

You can get that code by scrolling down to the Code Examples and clicking the Python tab.

Go ahead and try another classifier of your own choosing.

In [ ]:
### REPLACE THIS WITH YOUR CODE

## 3.3 Writing your own classifier

Now let's create a new classifier.  This one's pretty primitive:  we're going to train it on two movie reviews and then ask it to classify two other reviews.  

In [ ]:
from monkeylearn import MonkeyLearn
from time import sleep

# Use the API key from your account
ml = MonkeyLearn(MONKEYLEARN_API_KEY)

# Create a new classifier
res = ml.classifiers.create('Test Classifier')

# Get the id of the new module
module_id = res.result['classifier']['hashed_id']

# Get the id of the root node
res = ml.classifiers.detail(module_id)
root_id = res.result['sandbox_categories'][0]['id']

# Create two new categories on the root node
res = ml.classifiers.categories.create(module_id, 'Negative', root_id)
negative_id = res.result['category']['id']
res = ml.classifiers.categories.create(module_id, 'Positive', root_id)
positive_id = res.result['category']['id']

# Now let's upload some samples
samples = [('The movie was terrible, I hated it.', negative_id), ('I love this movie, I want to watch it again!', positive_id)]
res = ml.classifiers.upload_samples(module_id, samples)

# Now let's train the module!
res = ml.classifiers.train(module_id)

# Now you should wait till the module has completed training, it will take a couple of minutes
sleep(20)

# Classify some texts
res = ml.classifiers.classify(module_id, ['I love the movie', 'I hate the movie'], sandbox=True)
print(res.result)

Run the above code and take a look at the output. If you get an error that looks like
`MonkeyLearnException: Error: ["Max custom modules exceeded"]`
you will need to go to your MonkeyLearn dashboard and delete the module.  Alternatively, you can do it from here with:

`ml.classifiers.delete(module_id)`

Go ahead and try to classify some additional bits of text:

In [ ]:
res = ml.classifiers.classify(module_id, ['TRY PUTTING SOME TEXT HERE', 'AND HERE', 'AND HERE'], sandbox=True)
print(res.result)

When you're done with that classifier, go ahead and delete it to make space for your next classifier:

In [ ]:
ml.classifiers.delete(module_id)

Your final task for this lab is to train and test a classifier based on some real data.  You should train it on at least 20 classified text samples.  You will need to label them yourself: you could do sentiment (positive/negative) or any other labelling system (e.g. language).  For data, you could use tweets (either via an API or just copying and pasting them into your code for the purposes of this lab), comments from a blog, or any other source of text that you can think of.

Once you've created and trained your classifier, test it on 3 other text samples and try to assess how "good" your classifier is.

In [ ]:
### REPLACE THIS WITH YOUR CODE (HINT: COPY THE SAMPLE CLASSIFIER ABOVE AS A STARTING POINT)